# 第一次运行

在本节中，我们将演示 XiangShan 最简单的编译和运行流程。

xs-env 仓库提供了编译和运行 XiangShan 所需的环境配置脚本，您可以直接从 github 克隆该仓库。

In [ ]:
%%bash
# 本次 tutorial 中，我们已经配置好本地目录，故不需要执行这些命令，以下命令供参考
# 克隆仓库
# git clone https://github.com/OpenXiangShan/xs-env.sh 
# 进入 xs-env
# cd xs-env
# 切换分支
# git checkout -b tutorial-2025 origin/tutorial-2025
# 安装依赖，你可以修改使用不同的包管理工具
# sudo -s ./setup-tools.sh
# 准备工具，测试开发环境
# source setup.sh
# 更新子模块仓库
# ./update-submodule.sh

随后我们就可以开始开发了。

XiangShan 的编译和运行依赖一些环境变量，xs-env 的 `env.sh` 脚本会帮我们配置好。每开启一个新终端都需要执行该脚本，您可以将其配置到您的 `.bashrc` 配置文件中。如 00-welcome 一节所述，在本次 tutorial 中，每个单元格都相当于一个新的 bash 环境，故每个单元格都需要重新执行。

In [ ]:
%%bash
cd ../ && source env.sh

env | grep _HOME

配置好环境变量以后，我们可以直接进入 `${NOOP_HOME}`（指向 `xs-env/XiangShan`）来编译 XiangShan。

编译参数我们将在后面进行介绍。

同时我们还可以使用 tree 来查看项目结构

In [ ]:
%%bash
tree -d -L 1 ..

香山提供了数百个可自由定制的参数。其中，

- `src/main/scala/top/Configs.scala` 定义了处理器核参数
- SoC 参数通过 yaml 文件在编译时传递，`src/main/resources/config/Default.yaml` 是默认配置

您可以通过 Ctrl+P 快捷键打开文件搜索，输入上述文件后可以快速跳转

确定好配置后，我们就可以开始编译香山，获取 Verilog 和仿真程序了

In [ ]:
%%bash
cd .. && source env.sh
cd ${NOOP_HOME}

# 注意⚠️：编译 XiangShan 非常占用计算资源，在本次 Tutorial 中请使用编译好的二进制文件，不要真的执行下面的命令
# 参考配置：16 核，64GB 物理内存
# make emu -j16 CONFIG=MinimalConfig

# 其他编译选项
# CONFIG=MinimalConfig  香山的配置
# EMU_THREADS=4         仿真线程数
# EMU_TRACE=1           启用波形
# WITH_DRAMSIM=1        使用 DRAMSim3 模拟 DRAM
# WITH_CHISELDB = 1     启用 ChiselDB 特性
# WITH_CONSTANTIN = 1   启用 Constantin 特性

以上命令会生成 build/emu 和 build/rtl 等文件

- build/rtl/*.sv 是使用 chisel 编译出的 verilog 文件
- build/emu 是进一步使用 verilator 编译出的仿真可执行文件

我们可以直接运行 `./build/emu` 对 XiangShan 进行仿真，运行 riscv64 的程序。

在本次 tutorial 中我们没有实际编译 emu，因此改为运行 `${ASSET_DIR}/precompile` 目录下预先编译好的 emu

运行参数我们将在后面进行介绍。

In [ ]:
%%bash
cd .. && source env.sh
cd ${NOOP_HOME}

$(get_asset emu-precompile/emu) \
    -i $(get_asset workload/hello-riscv64-xs.bin) \
    --no-diff \
    2>/dev/null

# 一些关键运行时参数
# -i                        待运行的负载
# -C / -I                   最大周期数 / 最大指令数
# --diff=PATH / --no-diff   参考模型的路径 / 关闭 difftest

需要注意，XiangShan 会在运行结束后向 stderr 输出性能计数器信息，文本量非常大，因此无论何时都建议将 stderr 重定向到某一个文件。在上面的示例中，我们不关心性能计数器，因此将其重定向到 `/dev/null`，您可以按需修改。